In [ ]:
from fastai.text.all import *

# Data

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

(#2) [Path('/home/renato/.fastai/data/imdb_sample/texts.csv'),Path('/home/renato/.fastai/data/imdb_sample/models')]

In [ ]:
df = pd.read_csv(path/'texts.csv')
df.sample(3)

,label,text,is_valid
835,positive,"This review contains spoilers for those who are not aware of the details of the true story on which this movie is based.<br /><br />The right to be presumed ""Innocent until proven guilty"" is a basic entitlement of anyone in a civilised society; but according to Fred Schepisi's partisan but sadly convincing story of a famous Australian murder trial, it was not granted to Lindy Chamberlain, accused of killing her baby. The story suggesting her innocence was unlikely (a dingo was alleged to have taken it), but those implying her guilt even more so, and there was no solid evidence against her....",True
926,positive,"This is a movie that should be seen by everyone if you want to see great acting. Mr. Torn and Ms Farrel do an outstanding job. I think they should have it on TV again so a new audience can enjoy it. Wonderful performances.<br /><br />It gives you a real feel of what the pioneers had to go through both physically and emotionally. Great unheard of movie.<br /><br />It was done when Ms. Farrel was very young. I had always thought of her as a comedian, but this certainly is not a comedy and she is just wonderful. There is very little dialogs, but that just make it seem more real. Mr. Torn as a...",True
424,negative,"I live in Salt Lake City and I'm not a Mormon, so why did I rent this movie? Well because I live in Utah and thought it'd be nice to see locations I know in a film. I really knew going into it that I wasn't going to get the inside jokes so I wasn't surprised when I sat with the deer in the headlights stare. What I was surprised at was the ant-non Mormon actions that were placed in this film.<br /><br />I know it's a Mormon film, catered to the members of the LDS Church, but I found it offensive because of the typical stereotype of people that isn't of their faith. Every non Mormon, which w...",False


In [ ]:
dls = TextDataLoaders.from_df(df, text_col='text', is_lm=True, valid_pct=0.1)

In [ ]:
dls.show_batch(max_n=3)

,text,text_
0,"xxbos xxmaj okay , i was bored and decided to see this movie . xxmaj but i think the main thing that brought this movie down was that there would be a hour of footage , then basically that same hour repeated 4 times . xxmaj it consists of 1 . xxmaj gathering the troops and discussing the attack plan , 2 . xxmaj flashbacks to the men 's wives 3 .","xxmaj okay , i was bored and decided to see this movie . xxmaj but i think the main thing that brought this movie down was that there would be a hour of footage , then basically that same hour repeated 4 times . xxmaj it consists of 1 . xxmaj gathering the troops and discussing the attack plan , 2 . xxmaj flashbacks to the men 's wives 3 . xxmaj"
1,"especially the close - up shots where the details of the giant ( for those times ) 4 - engine xxunk ( then 18 , xxrep 3 0 + xxunk , now nearly xxunk ) can be seen . xxmaj good insight into the different levels of training that a pilot - xxunk went through on his way to being assigned to a xxunk crew ( of course , xxup very gender","the close - up shots where the details of the giant ( for those times ) 4 - engine xxunk ( then 18 , xxrep 3 0 + xxunk , now nearly xxunk ) can be seen . xxmaj good insight into the different levels of training that a pilot - xxunk went through on his way to being assigned to a xxunk crew ( of course , xxup very gender -"
2,"to care . xxmaj then it rushes through a resolution of all the outstanding problems in about a minute of screen time leaving the viewer feeling like they have just wasted their time . xxbos xxmaj if you like occasional nudity with junior high school level slap stick comedy , then look no further . \n\n xxmaj starting at about the halfway point , the beautiful and erotic xxmaj xxunk xxmaj xxunk","care . xxmaj then it rushes through a resolution of all the outstanding problems in about a minute of screen time leaving the viewer feeling like they have just wasted their time . xxbos xxmaj if you like occasional nudity with junior high school level slap stick comedy , then look no further . \n\n xxmaj starting at about the halfway point , the beautiful and erotic xxmaj xxunk xxmaj xxunk starts"


## Model

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, metrics=[accuracy, perplexity], path=path, wd=0.1).to_fp16()

In [ ]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(7080, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(7080, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0-2): 3 x RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=7080, bias=True)
    (output_dp): RNNDropout()
  )
)

## Train

In [ ]:
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.406633,3.977496,0.277549,53.383190,00:04
1,4.153448,3.931969,0.281024,51.007298,00:04


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.957479,3.891485,0.284667,48.983585,00:05
1,3.903596,3.896640,0.284670,49.236752,00:05
2,3.803774,3.907343,0.282750,49.766567,00:05
3,3.649792,3.912997,0.282960,50.048725,00:05
4,3.479743,3.947564,0.283476,51.808987,00:05
5,3.303140,3.984378,0.281126,53.751865,00:05
6,3.143640,4.029212,0.279696,56.216568,00:05
7,2.997500,4.061234,0.276563,58.045921,00:05
8,2.897017,4.083318,0.277235,59.342052,00:05
9,2.845984,4.089329,0.276851,59.699833,00:05


In [ ]:
learn.save_encoder('lm_encoder')

In [ ]:
inp = 'The movie was not very'
samples = 3
for _ in range(samples):
    res = learn.predict(inp, 40, temperature=0.75, no_bar=True)
    print(f'- {res}')

- The movie was not very good . When it was shown , it was very different from the start . It had the same amount of humor , humor and humor as the first part , but did it get worse .
- The movie was not very good . The movie was mostly predictable , with predictable plot twists , lack of plot , or lots of boring plot . It got really boring and the movie did n't work . 

 The story
- The movie was not very bad . The actors and actresses were terrible . The script was terrible , the dialogue was horrible , and the dialog was poor . The dialogue was awful and the script was poorly edited .
